In [47]:
import gzip
import json
import glob
import random
import tensorflow as tf
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from reddit.models import BatchTransformerForContextMLM
from transformers import TFDistilBertForMaskedLM, DistilBertTokenizer
from reddit.utils import load_tfrecord, mask_and_stack_mlm

In [48]:
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [49]:
pattern_1 = '../logs/mlm/10context_biencoder_3_1_attention/*/metrics/*/mlm/*/log.json'
fs = glob.glob(pattern_1)

In [50]:
colors = dict(zip([f'epoch-{e}' for e in range(5)],
                  ['darkgrey', 'lightblue', 
                   'darkblue', 'midnightblue', 
                   'black']))

In [ ]:
c = 0
fg, ax = plt.subplots(figsize=(20,10))
for idx, f in enumerate(fs):
    
    d = json.load(open(f))
    avg_loss = np.nanmean(d['test_losses'])
    avg_entropy = np.nanmean(d['test_entropy'])
    avg_true = np.nanmean(d['test_is_true_top'])
    # Get ds of test metrics
    sd_loss = np.nanstd(d['test_losses'])
    sd_entropy = np.nanstd(d['test_entropy'])
    sd_true = np.nanstd(d['is_true_top'])
    # Get other metrics with rolling average
    loss = np.nanmean(d['losses'])
    entropy  = np.nanmean(d['entropy'])
    true = np.nanmean(d['is_true_top'])
    epoch = f.split('/')[-2]

    dict_log = {'epoch':[epoch], 
                'grouping': [f.split('/')[4]],
                 'test_loss': [avg_loss],
                 'test_entropy': [avg_entropy],
                 'test_true': [avg_true],
                 'test_sd_loss': [sd_loss],
                 'test_sd_entropy': [sd_entropy],
                 'test_sd_true': [sd_true],
                 'loss': [loss],
                 'entropy': [entropy],
                 'true': [true],
                 'model_id': f.split('/')[-4],
                 'folder': f.split('/')[3]}

    sns.lineplot(x=range(len(np.arange(0, 1000000, 2000))), 
                 y=[np.mean(d['is_true_top'][i:i+2000]) for i in np.arange(0, 1000000, 2000)],
                 label='_'.join([f.split('/')[4], f.split('/')[-4]]),
                 legend=True, c=colors[epoch])
    plt.axhline(y=avg_true, c=colors[epoch], linestyle='--')
    
    # Make into df
    df = pd.DataFrame(dict_log)
    if c == 0:
        adf = df 
    else:
        adf = pd.concat([adf, df], ignore_index=True, sort=True)
    c+=1

<ipython-input-51-c70f41e91249>:6: RuntimeWarning: Mean of empty slice
  avg_loss = np.nanmean(d['test_losses'])
<ipython-input-51-c70f41e91249>:7: RuntimeWarning: Mean of empty slice
  avg_entropy = np.nanmean(d['test_entropy'])
<ipython-input-51-c70f41e91249>:8: RuntimeWarning: Mean of empty slice
  avg_true = np.nanmean(d['test_is_true_top'])
/home/roberta/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/roberta/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/roberta/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/roberta/.local/lib/python3.8/site-packages/seaborn/relational.py:514: MatplotlibDeprecationWarning: Saw kwarg